In [1]:
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = ""
from keras.models import Model, Sequential
from keras.layers import Input, Convolution2D, ZeroPadding2D, MaxPooling2D, Flatten, Dense, Dropout, Activation, GlobalAveragePooling2D
from PIL import Image
import numpy as np
from keras.preprocessing.image import load_img, save_img, img_to_array
from keras.applications.imagenet_utils import preprocess_input
from keras.preprocessing import image
import matplotlib.pyplot as plt


def preprocess_image(image_path):
    img = load_img(image_path, target_size=(224, 224))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    return img


Using TensorFlow backend.


In [2]:

print("Constructing model")
model = Sequential()
model.add(ZeroPadding2D((1, 1), input_shape=(224, 224, 3)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, (3, 3), activation='relu',name='last_feature_layer'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(Convolution2D(4096, (7, 7), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(4096, (1, 1), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(2622, (1, 1)))
model.add(Flatten())
model.add(Activation('softmax'))
# you can download the pretrained weights from the following link
# https://drive.google.com/file/d/1CPSeum3HpopfomUEK1gybeuIVoeJT_Eo/view?usp=sharing
# or you can find the detailed documentation https://sefiks.com/2018/08/06/deep-face-recognition-with-keras/

from keras.models import model_from_json
print("loading weights")
model.load_weights('vgg_face_weights.h5')
print("weights loaded")


Constructing model
loading weights


weights loaded


In [34]:
# pad_model = Model(inputs=model.layers[0].input, outputs=model.get_layer('last_feature_layer').output)
x = GlobalAveragePooling2D()(model.get_layer('last_feature_layer').output)
x = Dense(100,activation='relu')(x)
x = Dense(80,activation='relu')(x)
x = Dense(1,activation='tanh')(x)
pad_model = Model(inputs=model.layers[0].input, outputs=x)


for layer in pad_model.layers:
    layer.trainable = False
pad_model.layers[-1].trainable = True
pad_model.layers[-2].trainable = True
pad_model.layers[-3].trainable = True

pad_model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_1_input (Inpu (None, 224, 224, 3)       0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 226, 226, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 224, 224, 64)      1792      
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 226, 226, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
zero_padding2d_3 (ZeroPaddin (None, 114, 114, 64)      0         
__________

In [ ]:
# model.summary()
# vgg_face_descriptor = Model(inputs=model.layers[0].input, outputs=model.layers[-2].output) # Functional model
# vgg_face_descriptor.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_1_input (Inpu (None, 224, 224, 3)       0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 226, 226, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 224, 224, 64)      1792      
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 226, 226, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
zero_padding2d_3 (ZeroPaddin (None, 114, 114, 64)      0         
__________

In [5]:
import os

def getTrainingDataReplay():
    train_imgs = []
    labels = []
    for file in os.listdir("datasets/ReplayAttackIMGs/attack_hand"):
        train_imgs.append(preprocess_image("datasets/ReplayAttackIMGs/attack_hand/" + file))
        labels.append(-1)
    for file in os.listdir("datasets/ReplayAttackIMGs/real"):
        train_imgs.append(preprocess_image("datasets/ReplayAttackIMGs/real/" + file))
        labels.append(1)
    train_imgs = np.array(train_imgs)
    train_imgs = train_imgs[:, 0, :, :]
    labels = np.array(labels)
    return train_imgs, labels



def getTestDataReplay():
    test_imgs = []
    labels = []
    for file in os.listdir("datasets/ReplayAttackIMGs_test/attack_hand"):
        test_imgs.append(preprocess_image("datasets/ReplayAttackIMGs_test/attack_hand/" + file))
        labels.append(-1)
    for file in os.listdir("datasets/ReplayAttackIMGs_test/real"):
        test_imgs.append(preprocess_image("datasets/ReplayAttackIMGs_test/real/" + file))
        labels.append(1)
    test_imgs = np.array(test_imgs)
    test_imgs = test_imgs[:, 0, :, :]
    labels = np.array(labels)
    return test_imgs, labels

#------------------------------------------------------------------------------------------------------
#------------------------------------------------------------------------------------------------------

def getTrainingDataOulu():
    train_imgs = []
    labels = []
    for file in os.listdir("datasets/OuluIMGs/attack"):
        train_imgs.append(preprocess_image("datasets/OuluIMGs/attack/" + file))
        labels.append(-1)
    for file in os.listdir("datasets/OuluIMGs/real"):
        train_imgs.append(preprocess_image("datasets/OuluIMGs/real/" + file))
        labels.append(1)
    train_imgs = np.array(train_imgs)
    train_imgs = train_imgs[:, 0, :, :]
    labels = np.array(labels)
    return train_imgs, labels



def getTestDataOulu():
    test_imgs = []
    labels = []
    for file in os.listdir("datasets/OuluIMGs_test/attack"):
        test_imgs.append(preprocess_image("datasets/OuluIMGs_test/attack/" + file))
        labels.append(-1)
    for file in os.listdir("datasets/OuluIMGs_test/real"):
        test_imgs.append(preprocess_image("datasets/OuluIMGs_test/real/" + file))
        labels.append(1)
    test_imgs = np.array(test_imgs)
    test_imgs = test_imgs[:, 0, :, :]
    labels = np.array(labels)
    return test_imgs, labels

#------------------------------------------------------------------------------------------------------
#------------------------------------------------------------------------------------------------------

def getTrainingDataOuluP1():
    train_imgs = []
    labels = []
    for file in os.listdir("datasets/Oulu_p1_IMGs/attack"):
        train_imgs.append(preprocess_image("datasets/Oulu_p1_IMGs/attack/" + file))
        labels.append(-1)
    for file in os.listdir("datasets/Oulu_p1_IMGs/real"):
        train_imgs.append(preprocess_image("datasets/Oulu_p1_IMGs/real/" + file))
        labels.append(1)
    train_imgs = np.array(train_imgs)
    train_imgs = train_imgs[:, 0, :, :]
    labels = np.array(labels)
    return train_imgs, labels



def getTestDataOuluP1():
    test_imgs = []
    labels = []
    for file in os.listdir("datasets/Oulu_p1_IMGs_test/attack"):
        test_imgs.append(preprocess_image("datasets/Oulu_p1_IMGs_test/attack/" + file))
        labels.append(-1)
    for file in os.listdir("datasets/Oulu_p1_IMGs_test/real"):
        test_imgs.append(preprocess_image("datasets/Oulu_p1_IMGs_test/real/" + file))
        labels.append(1)
    test_imgs = np.array(test_imgs)
    test_imgs = test_imgs[:, 0, :, :]
    labels = np.array(labels)
    return test_imgs, labels

In [36]:
tr_imgs = 0
test_imgs = 0

In [37]:

tr_imgs, tr_labels = getTrainingDataReplay()
# test_imgs, test_labels = getTestDataReplay()

# tr_imgs, tr_labels = getTrainingDataOuluP1()
test_imgs, test_labels = getTestDataOuluP1()

print(tr_imgs.shape, tr_labels.shape)
print(test_imgs.shape, test_labels.shape)


(2400, 224, 224, 3) (2400,)
(1747, 224, 224, 3) (1747,)


In [41]:
# test_imgs, test_labels = getTestDataOuluP1()
test_imgs, test_labels = getTestDataReplay()
print(test_imgs.shape, test_labels.shape)

(3200, 224, 224, 3) (3200,)


In [38]:
from keras.optimizers import Adam
from keras.losses import binary_crossentropy
from keras.metrics import binary_accuracy

# model.compile(optimizer='adam', loss='mse', metrics=['accuracy',binary_accuracy])
pad_model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])


In [39]:
history = pad_model.fit(tr_imgs, tr_labels, batch_size=16, epochs=2)
print(history.history)
# pad_model.save("padModel2_Oulu_p1_19122018.h5")

Epoch 1/2


  16/2400 [..............................] - ETA: 46s - loss: 1.1282 - acc: 0.3750

  32/2400 [..............................] - ETA: 29s - loss: 1.2220 - acc: 0.3125

  48/2400 [..............................] - ETA: 22s - loss: 1.1320 - acc: 0.3125

  64/2400 [..............................] - ETA: 19s - loss: 0.9714 - acc: 0.3750

...

2352/2400 [============================>.] - ETA: 0s - loss: 0.0062 - acc: 0.9962

2368/2400 [============================>.] - ETA: 0s - loss: 0.0061 - acc: 0.9962

2384/2400 [============================>.] - ETA: 0s - loss: 0.0061 - acc: 0.9962

2400/2400 [==============================] - 13s 5ms/step - loss: 0.0061 - acc: 0.9962


{'loss': [0.09370354033211697, 0.006060124621693603], 'acc': [0.93125, 0.99625]}


In [ ]:
# history = model.fit(tr_imgs, tr_labels, batch_size=1, epochs=2)
# print(history.history)
# model.save("MyVGGface.h5")

In [1]:
from keras.models import load_model
# model = load_model("MyVGGface.h5")
pad_model = load_model("padModel.h5")

Using TensorFlow backend.


In [42]:
score = pad_model.evaluate(test_imgs,test_labels,batch_size=16 )
print(score)

  16/3200 [..............................] - ETA: 14s

  32/3200 [..............................] - ETA: 15s

  48/3200 [..............................] - ETA: 14s

  64/3200 [..............................] - ETA: 14s

  80/3200 [..............................] - ETA: 14s

...

3152/3200 [============================>.] - ETA: 0s

3168/3200 [============================>.] - ETA: 0s

3184/3200 [============================>.] - ETA: 0s

3200/3200 [==============================] - 14s 5ms/step


[0.2172800602152933, 0.910625]


In [8]:
scores = pad_model.predict(test_imgs,batch_size=16,verbose=1)

  16/3200 [..............................] - ETA: 29s

  32/3200 [..............................] - ETA: 22s

  48/3200 [..............................] - ETA: 19s

  64/3200 [..............................] - ETA: 18s

  80/3200 [..............................] - ETA: 17s

...

3136/3200 [============================>.] - ETA: 0s

3152/3200 [============================>.] - ETA: 0s

3168/3200 [============================>.] - ETA: 0s

3184/3200 [============================>.] - ETA: 0s

3200/3200 [==============================] - 15s 5ms/step


In [ ]:
np.savetxt("scores/scores.txt",scores)
np.savetxt("scores/true_labels.txt",test_labels)

In [ ]:
print(scores)

[[-0.99516386]
 [-0.7635757 ]
 [-0.99999994]
 ...
 [ 0.99604744]
 [ 0.9993456 ]
 [ 0.99867606]]
